In [201]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
#import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [25]:
from datasets import load_dataset

ds = load_dataset("traversaal-ai-hackathon/hotel_datasets")

In [202]:
import pandas as pd
df = pd.DataFrame(ds['train'])

In [203]:
df.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


In [60]:
hotel_reviews = {}

for index, row in df.iterrows():
    hotel_name = row['hotel_name']
    review_title = row['review_title']
    
    if hotel_name not in hotel_reviews:
        hotel_reviews[hotel_name] = []
    
    hotel_reviews[hotel_name].append(review_title)

# Save the dictionary
with open('hotel_reviews_dict.json', 'w') as f:
    json.dump(hotel_reviews, f)


In [61]:
hotel_reviews_keys = hotel_reviews.keys()
print(hotel_reviews_keys)

dict_keys(['Romance Istanbul Hotel', 'Hotel Sultania', 'Conrad Istanbul Bosphorus', 'Hilton Istanbul Bomonti Hotel & Conference Center', 'Hotel Yasmak Sultan', 'Radisson Blu Hotel Istanbul Ottomare', 'Demiray Hotel&Spa', 'White House Hotel Istanbul', 'Swissotel The Bosphorus, Istanbul', 'Primero Hotel', 'The Story Hotel Pera', 'Hotel Amira Istanbul', 'Mula Hotel', 'Sirkeci Mansion', 'Tomtom Suites', 'Sura Hagia Sophia Hotel', 'Ramada by Wyndham Istanbul Old City', 'MySuite Istanbul', 'Wyndham Istanbul Oldcity', 'Cheers Lighthouse', 'Crowne Plaza Istanbul - Florya, an IHG hotel', 'AJWA Sultanahmet', 'Skalion Hotel & Spa', 'Pera Palace Hotel', 'Richmond Istanbul', 'Wyndham Grand Istanbul Kalamis Marina Hotel', 'YOTEL Istanbul Airport Hotel (Landside)', 'InterContinental Istanbul', 'Pell Palace Hotel Spa', 'Elite World Grand İstanbul Basın Ekspres', 'Staypineapple - An Elegant Hotel', 'Beacon Grand A Union Square Hotel', 'Hotel Riu Plaza Fisherman’s Wharf', 'Hotel Emblem San Francisco', '

In [105]:
import openai
import json

openai.api_key = 'OPENAI_TOKEN'

summary_dict = {}

for hotel_name, review_titles in hotel_reviews.items():
    combined_text = " ".join(review_titles)
    prompt = f"Please summarize the following reviews for {hotel_name}: {combined_text}"
    
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",  # or use "gpt-4" if available
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
      ],
      max_tokens=100,
      temperature=0.5
    )
    
    summary = response['choices'][0]['message']['content'].strip()
    print(f"Summary for {hotel_name}: {summary}")
    
    summary_dict[hotel_name] = summary

# Save the summary_dict to a JSON file
with open('summary_output.json', 'w') as json_file:
    json.dump(summary_dict, json_file, indent=4)

Summary for Romance Istanbul Hotel: The reviews for Romance Istanbul Hotel are overwhelmingly positive, with guests praising the exceptional boutique hotel that offers great value for money. Many guests mentioned that the hotel exceeded all expectations and provided fantastic accommodations in a great location. The attention to detail and exquisite atmosphere of the hotel were highlighted, with guests describing their stay as perfect and lovely in every aspect. The staff was also highly praised for their friendliness and professionalism. Overall, guests highly recommended the hotel for a wonderful stay in Istanbul, with many mentioning that it
Summary for Hotel Sultania: The reviews for Hotel Sultania are overwhelmingly positive, with guests praising the excellent location, service, and quirky clever rooms. Many guests mention the hospitality and family-friendly atmosphere of the hotel, making it a popular choice for honeymoons, family trips, and special occasions. The hotel's spa, com

In [200]:
import re

df_combined = df.copy()

df_combined['summary'] = df_combined['summary'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

# Translate all the "combined" column to lower case.
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['summary']= df_combined['summary'].apply(lambda x: lower_case(x))


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3880/4255807732.py:5: SyntaxWarning: invalid escape sequence '\s'
  df_combined['summary'] = df_combined['summary'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))


In [108]:
df_combined.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,summary
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...


In [135]:
sample = df_combined[:1000]

In [137]:
embedder = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)


<All keys matched successfully>


In [138]:
# Switch to CPU
embedder =  embedder.to('cpu')

startTime = time.time()

# Create a column named 'embedding', where the 'combined' column is turned to embeddings by the model.
sample["embedding"] = sample.summary.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 135.70584154129028


/tmp/ipykernel_3880/2328924640.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.summary.apply(lambda x: embedder.encode(x))


In [139]:
sample

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,summary,embedding
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Richmond Istanbul,A timeless place where you have the famous Per...,Stayed at 208,None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,..."
996,Richmond Istanbul,A timeless place where you have the famous Per...,our home in Istanbul,None,NaN,January 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,..."
997,Richmond Istanbul,A timeless place where you have the famous Per...,A must stay!!!!,None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,..."
998,Richmond Istanbul,A timeless place where you have the famous Per...,Ever memorable Richmond stay for their dedicat...,None,NaN,March 2023,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,..."


In [140]:
sample.to_pickle('df.pkl')    #to save the dataframe, df to 123.pkl

In [141]:
df_with_embedding = pd.read_pickle('df.pkl') #to load 123.pkl back to the dataframe df

In [142]:
df_with_embedding.head()

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,summary,embedding
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993..."


In [143]:
query = 'Not worth the effort or money + This hotel is not worth the effort or the price'

# Embed the previous query.
query_embedding = embedder.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [144]:
# First element of the 'combined' column.
df_with_embedding.summary[0]

'the reviews for romance istanbul hotel are overwhelmingly positive with guests praising the hotel for being exceptional exceeding expectations and offering great value for money the hotel is described as having a fantastic location fantastic accommodations and paying attention to every detail guests highly recommend the hotel mentioning the friendly staff clean rooms and great location in the old city of istanbul the overall consensus is that romance istanbul hotel is a luxurious haven of hospitality and a great place to stay in istanbul'

In [145]:
# Create a list of embeddings, from the contents of the "combined" column.
corpus_embeddings = embedder.encode(df_with_embedding.summary,show_progress_bar=True)

# Create a column called 'similarity', displaying the cosine similarity between your embedded query and each embedded content from the 'corpus_embeddings' variable.
df_with_embedding['similarity']=cosine_similarity(corpus_embeddings, query_embedding)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [205]:
df_with_embedding

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,summary,embedding,similarity
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Richmond Istanbul,A timeless place where you have the famous Per...,Stayed at 208,None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,...",[0.3710443]
996,Richmond Istanbul,A timeless place where you have the famous Per...,our home in Istanbul,None,NaN,January 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,...",[0.3710443]
997,Richmond Istanbul,A timeless place where you have the famous Per...,A must stay!!!!,None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,...",[0.3710443]
998,Richmond Istanbul,A timeless place where you have the famous Per...,Ever memorable Richmond stay for their dedicat...,None,NaN,March 2023,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. N

In [206]:
df_with_embedding

,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,summary,embedding,similarity
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,the reviews for romance istanbul hotel are ove...,"[-1.0716542, 1.8314267, -3.5589826, -0.1963993...",[0.49455354]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Richmond Istanbul,A timeless place where you have the famous Per...,Stayed at 208,None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,...",[0.3710443]
996,Richmond Istanbul,A timeless place where you have the famous Per...,our home in Istanbul,None,NaN,January 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,...",[0.3710443]
997,Richmond Istanbul,A timeless place where you have the famous Per...,A must stay!!!!,None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. No: 227 Tünel,Istanbul,Türkiye,nan,"[0.26118433, 0.7683328, -1.921965, -0.8382798,...",[0.3710443]
998,Richmond Istanbul,A timeless place where you have the famous Per...,Ever memorable Richmond stay for their dedicat...,None,NaN,March 2023,https://www.tripadvisor.com/Hotel_Review-g2939...,https://dynamic-media-cdn.tripadvisor.com/medi...,$ (Based on Average Nightly Rates for a Standa...,4.5,1245,Istiklal Cad. N

In [207]:
def search(query):
    # Define a number of results to return, in this case, return only the first 15 results ranked by similarity.
    n = 15

    # Embed the query.
    query_embedding = embedder.encode(query)

    # Generate the similarity column, based on your query.
    df_with_embedding["similarity"] = df_with_embedding.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

    # Calculate the top 'n' most similar results by similarity.
    results = (
        df_with_embedding.sort_values("similarity", ascending=False)
        .head(n))
    resultlist = []
    

    # Display them in a very concise and ordered manner.
    hlist = []
    for r in results.index:
        if results.hotel_name[r] not in hlist:
            
            smalldf = results.loc[results.hotel_name == results.hotel_name[r]]
            if smalldf.shape[1] > 3:
                smalldf = smalldf[:3]

            
            resultlist.append(
            {
                "name": results.hotel_name[r],
                "score": smalldf.similarity[r][0],
                "rating": smalldf.rating_value.max(),
                "relevant_reviews": [smalldf.review_text[s] for s in smalldf.index]
            })
            hlist.append(results.hotel_name[r])
    return resultlist

In [208]:
search('good food with a great customer serveice')

[{'name': 'Hilton Istanbul Bomonti Hotel & Conference Center',
  'score': 0.60961825,
  'rating': 5.0,
  'relevant_reviews': [None,
   None,
   'Ceren , Huri, Emrah and Sinan in Exec lounge are just fantastic…nothing was too much for them.  Went out of their way to put a smile on  every guests face.  The food offering is fantastic, one of the best I’ve seen.  It is a great experience.  The views are also pretty fantastic.']}]

In [89]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

<All keys matched successfully>


In [84]:
startTime = time.time()

df_combined["embedding"] = df_combined.summary.apply(lambda x: embedder.encode(x))
executionTime = (time.time()- startTime)
print('Execution time in seconds: '+str(executionTime))

In [ ]:
# Turn the dataframe you have just created to pickle file, for later use.
df_combined.to_pickle('entire_data.pkl')

In [ ]:
# drive.mount() loads the contents from your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read the pickle file you entered earlier.
import pandas as pd
df = pd.read_pickle('entire_data.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
query = 'good quility food with amazing room servivce'

In [ ]:
def search(query):
  # Define a number of results to return, in this case, return only the first 15 results ranked by similarity.
  n = 15

  # Embed the query.
  query_embedding = embedder.encode(query)

  # Generate the similarity column, based on your query.
  df["similarity"] = df.summary.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  # Calculate the top 'n' most similar results by similarity.
  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  # Display them in a very concise and ordered manner.
  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "score": smalldf.similarity[r][0],
            "rating": smalldf.rating.max(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist




In [ ]:
query = 'good quility food with amazing room servivce'

In [ ]:
results = search_reviews(df,query,15)

In [ ]:
results

In [ ]:

import gradio as gr

def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          smallarr = smalldf.similarity[r].max()
          sm =smalldf.rating[r].mean()

          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "description":results.description[r],
            "relevance score": smallarr.tolist(),
            "rating": sm.tolist(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist

def greet(query):
    bm25 = search(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="json")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)

# COPY THE URL THAT APPEARS BELOW